In [1]:
rcv_data = "imei=358709051786584&rmc=$GNRMC,042450.00,A,2503.32937,N,12131.91940,E,1.155,,080722,,,A*6B,ST,G00414,3685,-84,1324,,M8,32,43pct,9.34,4G,3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76,WiFi:,21ec4118245595MOJ,3104d5902adef0Software_Lab,3ae8cc18fec4c8TOATECH,3b4a5f996b9b8dDIRECT-8d-HP M154 LaserJet,3e04bd88a4e822LLGPGuest,410c473dfe62b8shinfu0310,4354833a076307HATHANHNGUYEN,47001ff3c158ec「岡」的「Wi-Fi 網路」,476409800ee216LLT-Lab_2.4G,4b1c5f2b2507c3.1.Free Wi-Fi"

# 開始前

In [2]:
gnss_data = {}

gnss_data["gsm"] = dict()
gnss_data["gps"] = dict()

# Parse GSM

### imei

In [3]:
import re


group = rcv_data.split(",", 1)
match_list = re.findall("^imei=.+&", group[0])
gnss_data["gsm"]["imei"] = match_list[0][5:-1]
gnss_data

{'gsm': {'imei': '358709051786584'}, 'gps': {}}

### sen(視訊品質)

In [4]:
group = rcv_data.split(",")
gnss_data["gsm"]["sen"] = int(group[19][1:])
gnss_data

{'gsm': {'imei': '358709051786584', 'sen': 8}, 'gps': {}}

### telecom(回傳路由)

In [5]:
# 用 4G 做正規表達式
group = re.findall("\,4G\,.+\,WiFi\:", rcv_data)
cell_string = group[0][4:-6]

# 有可能附近沒有基地台
# if 'SEARCH' in cell_string:
#     return 

cell_main_info = cell_string.split(";")[0].split(",")
cell_secondary_info = cell_string.split(";")[1:]

gnss_data["gsm"]["telecom"] = '4G'
gnss_data

{'gsm': {'imei': '358709051786584', 'sen': 8, 'telecom': '4G'}, 'gps': {}}

### cell_string(基地台資訊)

In [6]:
gnss_data["gsm"]["cell_string"] = cell_string
gnss_data["gsm"]["cell_id"] = cell_string # remove after refactor correction 
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76'},
 'gps': {}}

### company(電信公司)

In [7]:
def __get_telecom_company(telecom_comp_code):
    if telecom_comp_code in ['92']:
        return "CHT"
    elif telecom_comp_code in ['01', '03']:
        return "FET"

gnss_data["gsm"]["company"] = __get_telecom_company(cell_main_info[6])
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT'},
 'gps': {}}

### global_id、pyhsical_id

In [8]:
gnss_data["gsm"]["global_id"] = cell_main_info[8]
gnss_data["gsm"]["physical_id"] = cell_main_info[9]
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22'},
 'gps': {}}

### rsrp(參考信號接收功率)

https://kknews.cc/news/5b5vk68.html

 <img src='./mod31/01.jpg'>

In [9]:
gnss_data["gsm"]["rsrp"] = cell_main_info[11]
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96'},
 'gps': {}}

### all_physical_id

In [10]:
gnss_data["gsm"]["all_physical_id"] = list(c_info.split(",")[4] for c_info in cell_secondary_info)
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6']},
 'gps': {}}

### tac

類型分配代碼

In [11]:
gnss_data["gsm"]["tac"] = cell_main_info[7]
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60'},
 'gps': {}}

### all_rsrp

In [12]:
gnss_data["gsm"]["all_rsrp"] = list(c_info.split(",")[2] for c_info in cell_secondary_info)
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {}}

# Parse GPS

<img src='./mod31/02.png'>

<img src='./mod31/03.png'>

### rmc(定位資訊)

In [13]:
group = rcv_data.split(",", 1)
match_list = re.findall("rmc=.+$", group[0])
gnss_data["gps"]["rmc"] = match_list[0][4:]
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC'}}

### iso_datetime(發送時間)

In [14]:
from datetime import datetime, timedelta

data_list = rcv_data.split(",")
raw_date = data_list[9]
raw_time = data_list[1]
if len(raw_date) == 6 and len(raw_time) == 9:
    year = int("20" + raw_date[4:])
    month = int(raw_date[2:4])
    day = int(raw_date[:2])
    hour = int(raw_time[:2])
    minute = int(raw_time[2:4])
    second = int(raw_time[4:6])
    naive = datetime(year, month, day, hour, minute, second)
    print(naive)
    if naive > datetime.utcnow() + timedelta(seconds=2):
        tag_id = data_list[14]
        log.warning(f'{tag_id} got Future datetime {naive} , fix to {datetime.utcnow().isoformat()[:19] + "Z"}')
    naive = naive if naive <datetime.utcnow() else datetime.utcnow()
    gnss_data["gps"]["iso_datetime"] = naive.isoformat()[:19] + "Z"
gnss_data

2022-07-08 04:24:50


{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC', 'iso_datetime': '2022-07-08T04:24:50Z'}}

### status(	定位狀態)

In [15]:
data_list = rcv_data.split(",")
if data_list[2] != "":
    gnss_data["gps"]["status"] = data_list[2]
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A'}}

### latitude(緯度)、longitude(經度)

In [16]:
def convert_gis_coordinate(coordinate):
    integer = int(coordinate / 100)
    decimal_point = (coordinate / 100) - integer
    new_decimal_point = (decimal_point * 100) / 60
    return integer + new_decimal_point

def handle_latitude(direction, value, gnss_data):
    if len(direction) == 1 and value != "":
        latitude = convert_gis_coordinate(float(value))
        if direction == "S":
            latitude = 0 - latitude
        gnss_data["gps"]["latitude"] = latitude

def handle_longitude(direction, value, gnss_data):
    if len(direction) == 1 and value != "":
        longitude = convert_gis_coordinate(float(value))
        if direction == "W":
            longitude = 0 - longitude
        gnss_data["gps"]["longitude"] = longitude
        
handle_latitude(data_list[4], data_list[3], gnss_data)
handle_longitude(data_list[6], data_list[5], gnss_data)
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A',
  'latitude': 25.055489499999997,
  'longitude': 121.53199000000002}}

### speed(速度)

In [17]:
def handle_speed(knot, gnss_data):
    if knot != "":
        gnss_data["gps"]["speed"] = float(knot) * 1.852

handle_speed(data_list[7], gnss_data)
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A',
  'latitude': 25.055489499999997,
  'longitude': 121.53199000000002,
  'speed': 2.13906}}

### direction(方向)

In [18]:
if data_list[8] != "":
    gnss_data["gps"]["direction"] = float(data_list[8])
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A',
  'latitude': 25.055489499999997,
  'longitude': 121.53199000000002,
  'speed': 2.13906}}

### correction(定位模式)

In [19]:
if data_list[12] != "":
    gnss_data["gps"]["correction"] = data_list[12][0:1]
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A',
  'latitude': 25.055489499999997,
  'longitude': 121.53199000000002,
  'speed': 2.13906,
  'correction': 'A'}}

### noise(訊噪比)

In [20]:
if data_list[20] != "":
    gnss_data["gps"]["noise"] = int(data_list[20])
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A',
  'latitude': 25.055489499999997,
  'longitude': 121.53199000000002,
  'speed': 2.13906,
  'correction': 'A',
  'noise': 32}}

### pdop(位置精度因子)

<img src='./mod31/05.PNG'>

<img src='./mod31/04.PNG'>

<img src='./mod31/06.PNG'>

In [21]:
def handle_pdop(pdop, gnss_data):
    if pdop != "":
        gnss_data["gps"]["pdop"] = float(pdop)
        
handle_pdop(data_list[22], gnss_data)
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A',
  'latitude': 25.055489499999997,
  'longitude': 121.53199000000002,
  'speed': 2.13906,
  'correction': 'A',
  'noise': 32,
  'pdop': 9.34}}

# Parse tag

In [22]:
gnss_data["battery"] = dict()
data_list = rcv_data.split(",")

### event(回傳事件)

In [23]:
if data_list[13] != "":
    gnss_data["event"] = data_list[13]
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A',
  'latitude': 25.055489499999997,
  'longitude': 121.53199000000002,
  'speed': 2.13906,
  'correction': 'A',
  'noise': 32,
  'pdop': 9.34},
 'battery': {},
 'event': 'ST'}

### tag_id(HA 編號)

In [24]:
if data_list[14] != "":
    gnss_data["tag_id"] = data_list[14]
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A',
  'latitude': 25.055489499999997,
  'longitude': 121.53199000000002,
  'speed': 2.13906,
  'correction': 'A',
  'noise': 32,
  'pdop': 9.34},
 'battery': {},
 'event': 'ST',
 'tag_id': 'G00414'}

### battery_mv(電池電壓)

In [25]:
def handle_pending_sms_or_mv(data, gnss_data):
    group = re.findall("^(?i)UR.+", data)
    if len(group) > 0:
        gnss_data["pending_sms"] = int(data[3:])
    else:
        gnss_data["battery"]["mv"] = int(data)

if data_list[15] != "":
    handle_pending_sms_or_mv(data_list[15], gnss_data)
gnss_data

<ipython-input-25-fe9f4b4b065d>:2: DeprecationWarning: Flags not at the start of the expression '^(?i)UR.+'
  group = re.findall("^(?i)UR.+", data)


{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A',
  'latitude': 25.055489499999997,
  'longitude': 121.53199000000002,
  'speed': 2.13906,
  'correction': 'A',
  'noise': 32,
  'pdop': 9.34},
 'battery': {'mv': 3685},
 'event': 'ST',
 'tag_id': 'G00414'}

### battery_ma(電池電流)

In [26]:
def handle_sent_sms_or_ma(data, gnss_data):
    group = re.findall("^(?i)MS.+", data)
    if len(group) > 0:
        gnss_data["sent_sms"] = int(data[3:])
    else:
        gnss_data["battery"]["ma"] = int(data)
        
if data_list[16] != "":
    handle_sent_sms_or_ma(data_list[16], gnss_data)
gnss_data

<ipython-input-26-08ea09e24d43>:2: DeprecationWarning: Flags not at the start of the expression '^(?i)MS.+'
  group = re.findall("^(?i)MS.+", data)


{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A',
  'latitude': 25.055489499999997,
  'longitude': 121.53199000000002,
  'speed': 2.13906,
  'correction': 'A',
  'noise': 32,
  'pdop': 9.34},
 'battery': {'mv': 3685, 'ma': -84},
 'event': 'ST',
 'tag_id': 'G00414'}

### battery_mah(電池剩餘電量)

In [27]:
def handle_caller_number_or_mah(data, gnss_data):
    group = re.findall("^(?i)CALL.+", data)
    group2 = re.findall("^(?i)SMS.+", data)

    if len(group) > 0:
        gnss_data["caller_number"] = data[5:]
    elif len(group2) > 0:
        gnss_data["sms"] = data
    else:
        gnss_data["battery"]["mah"] = int(data)
        
if data_list[17] != "":
    handle_caller_number_or_mah(data_list[17], gnss_data)
gnss_data

<ipython-input-27-98d59a97d49a>:2: DeprecationWarning: Flags not at the start of the expression '^(?i)CALL.+'
  group = re.findall("^(?i)CALL.+", data)
<ipython-input-27-98d59a97d49a>:3: DeprecationWarning: Flags not at the start of the expression '^(?i)SMS.+'
  group2 = re.findall("^(?i)SMS.+", data)


{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A',
  'latitude': 25.055489499999997,
  'longitude': 121.53199000000002,
  'speed': 2.13906,
  'correction': 'A',
  'noise': 32,
  'pdop': 9.34},
 'battery': {'mv': 3685, 'ma': -84, 'mah': 1324},
 'event': 'ST',
 'tag_id': 'G00414'}

### auto_count(Auto封數)

In [28]:
if data_list[18] != "":
    gnss_data["auto_count"] = int(data_list[18])
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A',
  'latitude': 25.055489499999997,
  'longitude': 121.53199000000002,
  'speed': 2.13906,
  'correction': 'A',
  'noise': 32,
  'pdop': 9.34},
 'battery': {'mv': 3685, 'ma': -84, 'mah': 1324},
 'event': 'ST',
 'tag_id': 'G00414'}

### voltage(電池電量百分比)

In [29]:
if data_list[21] != "":
    gnss_data["voltage"] = data_list[21]
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A',
  'latitude': 25.055489499999997,
  'longitude': 121.53199000000002,
  'speed': 2.13906,
  'correction': 'A',
  'noise': 32,
  'pdop': 9.34},
 'battery': {'mv': 3685, 'ma': -84, 'mah': 1324},
 'event': 'ST',
 'tag_id': 'G00414',
 'voltage': '43pct'}

### rcv_datetime(接收時間)

In [30]:
gnss_data["rcv_datetime"] = datetime.utcnow().isoformat() + "Z"
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A',
  'latitude': 25.055489499999997,
  'longitude': 121.53199000000002,
  'speed': 2.13906,
  'correction': 'A',
  'noise': 32,
  'pdop': 9.34},
 'battery': {'mv': 3685, 'ma': -84, 'mah': 1324},
 'event': 'ST',
 'tag_id': 'G00414',
 'voltage': '43pct',
 'rcv_datetime': '2022-07-21T03:28:27.941767Z'}

### wife(wifi 資訊)

In [31]:
def handle_wifi(rcv_data, gnss_data):
    group = re.findall("\,WiFi\:.+$", rcv_data)
    if len(group) > 0:
        gnss_data["wifi"] = group[0][7:]
        # MSSI(2字元、十進位)、MAC(12字元)、SSID(最多34字元)
        wifi_list = group[0][7:].split(',')
        gnss_data['wifi_list'] = [
            {'RSSI': str(int(i[0: 2], 16)), 'MAC':i[2: 14], 'SSID': i[14:]} for i in wifi_list]
        
handle_wifi(rcv_data, gnss_data)
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A',
  'latitude': 25.055489499999997,
  'longitude': 121.53199000000002,
  'speed': 2.13906,
  'correction': 'A',
  'noise': 32,
  'pdop': 9.34},
 'battery': {'mv': 3685, 'ma': -84, 'mah': 1324},
 'event': 'ST',
 'tag_id': 'G00414',
 'voltage': '43pct',
 'rcv_datetime': '2022-07-21T03:28:27.941767Z',
 'wifi': '21ec4118245595MOJ,3104d5902adef0Software_Lab,3ae8cc18fec4c8TOATECH,3b4a5f996b9b8dDIRECT-8d-HP M154 LaserJet,3e04bd88a4e

### raw(原始資料)

In [32]:
gnss_data['raw'] = rcv_data
gnss_data

{'gsm': {'imei': '358709051786584',
  'sen': 8,
  'telecom': '4G',
  'cell_string': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'cell_id': '3050,7,20,20,FDD,466,92,2B60,33E2722,22,27,-96,-13,NOCONN;3050,-12.9,-96,27,22,-64;3050,-16.6,-103,20,6,-76',
  'company': 'CHT',
  'global_id': '33E2722',
  'physical_id': '22',
  'rsrp': '-96',
  'all_physical_id': ['22', '6'],
  'tac': '2B60',
  'all_rsrp': ['-96', '-103']},
 'gps': {'rmc': '$GNRMC',
  'iso_datetime': '2022-07-08T04:24:50Z',
  'status': 'A',
  'latitude': 25.055489499999997,
  'longitude': 121.53199000000002,
  'speed': 2.13906,
  'correction': 'A',
  'noise': 32,
  'pdop': 9.34},
 'battery': {'mv': 3685, 'ma': -84, 'mah': 1324},
 'event': 'ST',
 'tag_id': 'G00414',
 'voltage': '43pct',
 'rcv_datetime': '2022-07-21T03:28:27.941767Z',
 'wifi': '21ec4118245595MOJ,3104d5902adef0Software_Lab,3ae8cc18fec4c8TOATECH,3b4a5f996b9b8dDIRECT-8d-HP M154 LaserJet,3e04bd88a4e